# Downloads e imports

In [1]:
from treino_avaliacao import treinar_avaliar
from Models.LSTM.LSTM import LSTMModel
from Models.GRU.GRU import GRUModel
import sys, os
sys.path.append('..')


RANDOM_SEED = 33

c:\Users\raafm\Pessoal\Ciencia-de-dados\Stock-Prices-Prediction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
treinar_avaliar(LSTMModel, "VALE3", n_trials= 50,file_to_save_model= f"VALE3_all_features.pt")

[I 2024-07-11 21:24:39,329] A new study created in memory with name: no-name-b86c3589-f7ff-4c5f-9967-f7d4ed2e9ddb
[I 2024-07-11 21:24:52,953] Trial 0 finished with value: 0.87233197295611 and parameters: {'hidden_size': 90, 'num_layers': 2, 'learning_rate': 0.002427092964014823}. Best is trial 0 with value: 0.87233197295611.
[I 2024-07-11 21:25:02,274] Trial 1 finished with value: 0.8015271719376054 and parameters: {'hidden_size': 69, 'num_layers': 3, 'learning_rate': 0.0011620595775471512}. Best is trial 0 with value: 0.87233197295611.
[I 2024-07-11 21:25:12,787] Trial 2 finished with value: 0.7078433354949508 and parameters: {'hidden_size': 17, 'num_layers': 4, 'learning_rate': 0.0010056988953143283}. Best is trial 0 with value: 0.87233197295611.
[I 2024-07-11 21:25:20,811] Trial 3 finished with value: 0.7141000116609872 and parameters: {'hidden_size': 27, 'num_layers': 3, 'learning_rate': 0.00031271108947951567}. Best is trial 0 with value: 0.87233197295611.
[I 2024-07-11 21:25:28,9

Number of finished trials:  50
Best trial:
  Value:  0.9064002590996288
  Params: 
    hidden_size: 104
    num_layers: 1
    learning_rate: 0.0029308294553194235
saving model on: Models\LSTM\VALE3_all_features.pt
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       187
           1       1.00      1.00      1.00       183

    accuracy                           1.00       370
   macro avg       1.00      1.00      1.00       370
weighted avg       1.00      1.00      1.00       370

              precision    recall  f1-score   support

           0       0.61      0.65      0.63        68
           1       0.44      0.40      0.42        47

    accuracy                           0.55       115
   macro avg       0.53      0.53      0.53       115
weighted avg       0.54      0.55      0.54       115



In [3]:
treinar_avaliar(GRUModel,  "VALE3", n_trials= 50,file_to_save_model= f"VALE3_all_features.pt")

[I 2024-07-11 21:30:39,363] A new study created in memory with name: no-name-dbf5668c-ae6e-4176-b4a2-0e63cb207895
[I 2024-07-11 21:30:45,522] Trial 0 finished with value: 0.8863154540731483 and parameters: {'hidden_size': 117, 'num_layers': 2, 'learning_rate': 0.0007049488657437623}. Best is trial 0 with value: 0.8863154540731483.
[I 2024-07-11 21:30:53,047] Trial 1 finished with value: 0.7779643541022241 and parameters: {'hidden_size': 43, 'num_layers': 3, 'learning_rate': 0.0005863693796327769}. Best is trial 0 with value: 0.8863154540731483.
[I 2024-07-11 21:31:00,511] Trial 2 finished with value: 0.6537299879476584 and parameters: {'hidden_size': 23, 'num_layers': 3, 'learning_rate': 0.00012367087617512465}. Best is trial 0 with value: 0.8863154540731483.
[I 2024-07-11 21:31:07,929] Trial 3 finished with value: 0.8475064686271583 and parameters: {'hidden_size': 41, 'num_layers': 3, 'learning_rate': 0.0009400846128018092}. Best is trial 0 with value: 0.8863154540731483.
[I 2024-07-1

Number of finished trials:  50
Best trial:
  Value:  0.9096153846153847
  Params: 
    hidden_size: 59
    num_layers: 1
    learning_rate: 0.0021397712761214755
saving model on: Models\GRU\VALE3_all_features.pt
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       187
           1       0.99      1.00      1.00       183

    accuracy                           1.00       370
   macro avg       1.00      1.00      1.00       370
weighted avg       1.00      1.00      1.00       370

              precision    recall  f1-score   support

           0       0.60      0.68      0.63        68
           1       0.42      0.34      0.38        47

    accuracy                           0.54       115
   macro avg       0.51      0.51      0.51       115
weighted avg       0.53      0.54      0.53       115



In [4]:
treinar_avaliar(LSTMModel, "PETR4", n_trials= 50,file_to_save_model= f"PETR4_all_features.pt")

[I 2024-07-11 21:35:57,840] A new study created in memory with name: no-name-a6c73878-e8f0-4ab4-9768-43ad8654ad49
[I 2024-07-11 21:36:07,640] Trial 0 finished with value: 0.5990638990638991 and parameters: {'hidden_size': 99, 'num_layers': 5, 'learning_rate': 0.0007967891371302398}. Best is trial 0 with value: 0.5990638990638991.
[I 2024-07-11 21:36:17,192] Trial 1 finished with value: 0.6282385796881169 and parameters: {'hidden_size': 116, 'num_layers': 5, 'learning_rate': 0.00023386433418226695}. Best is trial 1 with value: 0.6282385796881169.
[I 2024-07-11 21:36:25,025] Trial 2 finished with value: 0.689074755032202 and parameters: {'hidden_size': 95, 'num_layers': 3, 'learning_rate': 0.00022033213877181005}. Best is trial 2 with value: 0.689074755032202.
[I 2024-07-11 21:36:32,171] Trial 3 finished with value: 0.6700460199058297 and parameters: {'hidden_size': 52, 'num_layers': 3, 'learning_rate': 0.0003669017760435989}. Best is trial 2 with value: 0.689074755032202.
[I 2024-07-11 

Number of finished trials:  50
Best trial:
  Value:  0.8672805971712616
  Params: 
    hidden_size: 125
    num_layers: 1
    learning_rate: 0.003005889218854955
saving model on: Models\LSTM\PETR4_all_features.pt
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       169
           1       0.98      0.99      0.98       201

    accuracy                           0.98       370
   macro avg       0.98      0.98      0.98       370
weighted avg       0.98      0.98      0.98       370

              precision    recall  f1-score   support

           0       0.43      0.67      0.53        49
           1       0.59      0.35      0.44        66

    accuracy                           0.49       115
   macro avg       0.51      0.51      0.48       115
weighted avg       0.52      0.49      0.48       115



In [5]:
treinar_avaliar(GRUModel,  "PETR4", n_trials= 50,file_to_save_model= f"PETR4_all_features.pt")

[I 2024-07-11 21:41:39,917] A new study created in memory with name: no-name-0560cae6-4328-43f3-89d5-736c046729a8
[I 2024-07-11 21:41:45,262] Trial 0 finished with value: 0.8181064966843051 and parameters: {'hidden_size': 119, 'num_layers': 1, 'learning_rate': 0.008587963481672421}. Best is trial 0 with value: 0.8181064966843051.
[I 2024-07-11 21:41:51,786] Trial 1 finished with value: 0.8492642225094439 and parameters: {'hidden_size': 68, 'num_layers': 2, 'learning_rate': 0.005303528956512207}. Best is trial 1 with value: 0.8492642225094439.
[I 2024-07-11 21:41:59,906] Trial 2 finished with value: 0.6538828263439391 and parameters: {'hidden_size': 41, 'num_layers': 4, 'learning_rate': 0.0006957190717812635}. Best is trial 1 with value: 0.8492642225094439.
[I 2024-07-11 21:42:05,401] Trial 3 finished with value: 0.6918486065474017 and parameters: {'hidden_size': 111, 'num_layers': 1, 'learning_rate': 0.00021944480730291317}. Best is trial 1 with value: 0.8492642225094439.
[I 2024-07-11

Number of finished trials:  50
Best trial:
  Value:  0.8825245486221096
  Params: 
    hidden_size: 127
    num_layers: 1
    learning_rate: 0.001326120350154706
saving model on: Models\GRU\PETR4_all_features.pt
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       169
           1       0.98      0.97      0.98       201

    accuracy                           0.98       370
   macro avg       0.97      0.98      0.98       370
weighted avg       0.98      0.98      0.98       370

              precision    recall  f1-score   support

           0       0.40      0.73      0.52        49
           1       0.50      0.20      0.28        66

    accuracy                           0.43       115
   macro avg       0.45      0.47      0.40       115
weighted avg       0.46      0.43      0.38       115



# Diagnóstico

Temos um caso evidente de overfitting com todos os modelos em todos os datasets

Podemos tentar resolver isso reduzindo o número de features fortemente correlacionadas. Desta forma reduziremos a complexidade do nosso modelo.

Outra abordagem possível é adicionar dropout

Vá para treino_corrigido.ipynb para ver as melhorias no treinamento